## Q.1. チャネル入れ替え

画像を読み込み、BGRをRGBの順に入れ替えよ。

画像の赤成分を取り出すには、以下のコードで可能。
cv2.imread()関数ではチャネルがBGRの順になることに注意！
これで変数redにimori.jpgの赤成分のみが入る。

```python
import cv2
img = cv2.imread("imori.jpg")
red = img[:, :, 2].copy()
```

In [ ]:
!pip install opencv-python

In [ ]:
import cv2

# 画像の読み込み
img = cv2.imread("imori.jpg")

# それぞれの値を格納する
Red = img[:,:,2].copy()
Blue = img[:,:,0].copy()
Green = img[:,:,1].copy()

# サイズをコピーする
img_RGB = img

# 付け替える
img_RGB[:,:,0] = Red
img_RGB[:,:,1] = Green
img_RGB[:,:,2] = Blue

# 保存して確認
cv2.imwrite("training_IMG/training_01.png",img_RGB)

In [ ]:
%reset -f

## Q.2. グレースケール化

画像をグレースケールにせよ。
グレースケールとは、画像の輝度表現方法の一種であり下式で計算される。

Y = 0.2126 R + 0.7152 G + 0.0722 B

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg")

# グレースケール化
img_Y = 0.2126*img[:,:,2] + 0.7152*img[:,:,1] + 0.0722*img[:,:,0]

# 変数の型を unit8 にして次元をまとめる
img_Y = np.expand_dims(img_Y.astype(np.uint8), axis=-1)

# 保存して確認する
cv2.imwrite("training_IMG/training_02.png",img_Y)

In [ ]:
%reset -f

## Q.3. 二値化

画像を二値化せよ。
二値化とは、画像を黒と白の二値で表現する方法である。
ここでは、グレースケールにおいて閾値を128に設定し、下式で二値化する。

```bash
y = { 0 (if y < 128)
     255 (else) }
```

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg")

# グレースケール化
img_Y = 0.3*img[:,:,2] + 0.59*img[:,:,1] + 0.11*img[:,:,0]
img_Y = np.expand_dims(img_Y.astype(np.uint8), axis=-1)

# ２値化
img_Y[img_Y < 128] = 0
img_Y[img_Y >= 128] = 255

# 保存して確認する
cv2.imwrite("training_IMG/training_03.png",img_Y)

In [ ]:
%reset -f

## Q.4. 大津の二値化

大津の二値化を実装せよ。
大津の二値化とは判別分析法と呼ばれ、二値化における分離の閾値を自動決定する手法である。
これは**クラス内分散**と**クラス間分散**の比から計算される。

グレースケールの輝度値（ピクセルの値）のヒストグラムはこうなる。

```python
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('assets/imori.jpg')
gray = 0.2126 * img[..., 2] + 0.7152 * img[..., 1] + 0.0722 * img[..., 0]
plt.hist(gray.ravel(), bins=255, rwidth=0.8, range=(0, 255))
plt.xlabel('value')
plt.ylabel('appearance')
plt.show()
```

<img src="assets/histogram-Gray.jpg" width=400>

二値化はある値を境界にして、０か１にする方法だけど、

- 閾値t未満をクラス0, t以上をクラス1とする。
- w0, w1 ... 閾値tにより分離された各クラスの画素数の割合 (w0 + w1 = 1を満たす)
- S0^2, S1^2 ... 各クラスの画素値の分散
- M0, M1 ... 各クラスの画素値の平均値

<img src="assets/histogram-Gray-1.png" width=400>

とすると、

<img src="assets/otsu_binary_1.png" width=500>

となり、分離度Xは次式で定義される。

<img src="assets/otsu_binary_2.png" width=300>

<!--
```bash
クラス内分散 Sw^2 = w0 * S0^2 + w1 * S1^2
クラス間分散 Sb^2 = w0 * (M0 - Mt)^2 + w1 * (M1 - Mt)^2 = w0 * w1 * (M0 - M1) ^2
画像全体の画素の分散 St^2 = Sw^2 + Sb^2 = (const)
以上より、分離度は次式で定義される。
分離度 X = Sb^2 / Sw^2 = Sb^2 / (St^2 - Sb^2)
```
-->

となるので、

<img src="assets/otsu_binary_3.png" width=300>

<!--
```bash
argmax_{t} X = argmax_{t} Sb^2
```
-->


となる。すなわち、Sb^2 =  w0 * w1 * (M0 - M1) ^2 が最大となる、閾値tを二値化の閾値とすれば良い。


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg")

# グレースケール化
img_Y = 0.2126*img[:,:,2] + 0.7152*img[:,:,1] + 0.0722*img[:,:,0]
img_Y = np.expand_dims(img_Y.astype(np.uint8), axis=-1)

# 大津の２値化のための閾値探し
max_Sb2 = 0
max_t = 0
H, W, C = img.shape
for t in range(0,256):
    # 閾値以上か未満で2値化
    v0 = img_Y[np.where(img_Y < t)]
    v1 = img_Y[np.where(img_Y >= t)]

    # 各クラスの画素数の割合を算出
    w0 = len(v0) / (H*W)
    w1 = len(v1) / (H*W)

    # 各クラスの画素数の平均を算出
    m0 = np.mean(v0) if len(v0) > 0 else 0
    m1 = np.mean(v1) if len(v0) > 0 else 0

    # 判定式
    tmp_Sb2 = w0*w1*(m0-m1)**2
    if tmp_Sb2 > max_Sb2:
        max_Sb2 = tmp_Sb2
        max_t = t

print("最適な閾値は",max_t)

# 最適な閾値で２値化
img_Y[img_Y < max_t] = 0
img_Y[img_Y >= max_t] = 255

# 保存して確認する
cv2.imwrite("training_IMG/training_04.png",img_Y)

In [ ]:
%reset -f

## Q.5. HSV変換

HSV変換を実装して、色相Hを反転せよ。

HSV変換とは、**Hue(色相)**、**Saturation(彩度)**、**Value(明度)** で色を表現する手法である。

- Hue ... 色合いを0~360度で表現し、赤や青など色の種類を示す。 ( 0 <= H < 360) 色相は次の色に対応する。

```bash
赤 黄色  緑  水色  青  紫   赤
0  60  120  180 240 300 360
```

- Saturation ... 色の鮮やかさ。Saturationが低いと灰色さが顕著になり、くすんだ色となる。 ( 0<= S < 1)
- Value ... 色の明るさ。Valueが高いほど白に近く、Valueが低いほど黒に近くなる。 ( 0 <= V < 1)

RGB -> HSV変換は以下の式で定義される。

R,G,Bが[0, 1]の範囲にあるとする。

```bash
Max = max(R,G,B)
Min = min(R,G,B)

H =  { 0                            (if Min=Max)
       60 x (G-R) / (Max-Min) + 60  (if Min=B)
       60 x (B-G) / (Max-Min) + 180 (if Min=R)
       60 x (R-B) / (Max-Min) + 300 (if Min=G)
       
V = Max

S = Max - Min
```

HSV -> RGB変換は以下の式で定義される。

```bash
C = S

H' = H / 60

X = C (1 - |H' mod 2 - 1|)

(R,G,B) = (V - C) (1,1,1) + { (0, 0, 0)  (if H is undefined)
                              (C, X, 0)  (if 0 <= H' < 1)
                              (X, C, 0)  (if 1 <= H' < 2)
                              (0, C, X)  (if 2 <= H' < 3)
                              (0, X, C)  (if 3 <= H' < 4)
                              (X, 0, C)  (if 4 <= H' < 5)
                              (C, 0, X)  (if 5 <= H' < 6)
```
ここでは色相Hを反転(180を加算)し、RGBに直し画像を表示せよ。


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

# RGB -> HSV 変換
# 範囲を[0,1]内に収める
img_01 = img/255
# 正規化の後にUnit8型になるので、floatに直す
img_01 = img_01.astype(np.float32)

# 画像の中で最大と最小の値を算出
Max = np.max(img_01, axis=2).copy()
Min = np.min(img_01, axis=2).copy()

# HSV変換後の変数を用意
hsv = np.zeros_like(img_01, dtype=np.float32)

# HSV の値作成
hsv[:,:,1] = Max - Min
hsv[:,:,2] = Max

# どの色が最小なのかを記録
min_arg = np.argmin(img, axis=2)

hsv[:,:,0][np.where(Max == Min)] = 0
# Bが最小のところ
idx = np.where(min_arg==0)
hsv[:,:,0][idx] = 60*(img_01[:,:,1][idx] - img_01[:,:,2][idx])/(Max[idx] - Min[idx]) + 60

# Gが最小のところ
idx = np.where(min_arg==1)
hsv[:,:,0][idx] = 60*(img_01[:,:,2][idx] - img_01[:,:,0][idx])/(Max[idx] - Min[idx]) + 300

# Rが最小のところ
idx = np.where(min_arg==2)
hsv[:,:,0][idx] = 60*(img_01[:,:,0][idx] - img_01[:,:,1][idx])/(Max[idx] - Min[idx]) + 180

# 色相Hを反転(180を加算) + 範囲を[0,360]に抑える
hsv[:,:,0] = (hsv[:,:,0]+180)%360

# HSV -> RGB 変換
C = hsv[:,:,1]
H_ = hsv[:,:,0]/60
V = hsv[..., 2]
X = C*(1 - np.abs(H_%2 - 1))
Z = np.zeros_like(hsv[:,:,0])
output = np.zeros_like(hsv)

vals = [[Z,X,C], [Z,C,X], [X,C,Z], [C,X,Z], [C,Z,X], [X,Z,C]]

for i in range(6):
       ind = np.where((i <= H_) & (H_ < (i+1)))
       output[:,:,0][ind] = (V - C)[ind] + vals[i][0][ind]
       output[:,:,1][ind] = (V - C)[ind] + vals[i][1][ind]
       output[:,:,2][ind] = (V - C)[ind] + vals[i][2][ind]

# 元に戻す
output[np.where(Max==Min)] = 0
out = np.clip(output, 0, 1)
out = (out * 255).astype(np.uint8)

# 保存して確認する
cv2.imwrite("training_IMG/training_05.png", out)

In [ ]:
%reset -f

## Q.6. 減色処理

ここでは画像の値を256^3から4^3、すなわちR,G,B in {32, 96, 160, 224}の各4値に減色せよ。
これは量子化操作である。
各値に関して、以下の様に定義する。

```bash
val = {  32  (  0 <= val <  64)
         96  ( 64 <= val < 128)
        160  (128 <= val < 192)
        224  (192 <= val < 256)
```

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

# 現職処理を行う
img[:,:,:][np.where((0 <= img[:,:,:]) & (img[:,:,:] < 64))] = 32
img[:,:,:][np.where((64 <= img[:,:,:]) & (img[:,:,:] < 128))] = 96
img[:,:,:][np.where((128 <= img[:,:,:]) & (img[:,:,:] < 192))] = 160
img[:,:,:][np.where((192 <= img[:,:,:]) & (img[:,:,:] < 256))] = 224

# 保存して確認する
cv2.imwrite("training_IMG/training_06.png", img)

In [ ]:
%reset -f

## Q.7. 平均プーリング

ここでは画像をグリッド分割(ある固定長の領域に分ける)し、かく領域内(セル)の平均値でその領域内の値を埋める。
このようにグリッド分割し、その領域内の代表値を求める操作は**Pooling(プーリング)** と呼ばれる。
これらプーリング操作は**CNN(Convolutional Neural Network)** において重要な役割を持つ。

これは次式で定義される。

```bash
v = 1/|R| * Sum_{i in R} v_i
```

ここではimori.jpgは128x128なので、8x8にグリッド分割し、平均プーリングせよ。

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def avarage_pooling(img, grid):
    output = img.copy()
    H,W,C = img.shape

    # 幅を分割した時の数を算出
    W_ = int(W/grid)

    # 高さを分冊した時の数を算出
    H_ = int(H/grid)

    for x in range(W_):
        for y in range(H_):
            for c in range(C):
                output[grid*y:grid*(y+1), grid*x:grid*(x+1), c] = np.mean(output[grid*y:grid*(y+1), grid*x:grid*(x+1), c])
    return output

out = avarage_pooling(img, 8)

# 保存して確認する
cv2.imwrite("training_IMG/training_07.png", out)

In [ ]:
%reset -f

## Q.8. Maxプーリング

ここでは平均値でなく最大値でプーリングせよ。

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def max_pooling(img, grid):
    output = img.copy()
    H,W,C = img.shape

    # 幅を分割した時の数を算出
    W_ = int(W/grid)

    # 高さを分冊した時の数を算出
    H_ = int(H/grid)

    for x in range(W_):
        for y in range(H_):
            for c in range(C):
                output[grid*y:grid*(y+1), grid*x:grid*(x+1), c] = np.max(output[grid*y:grid*(y+1), grid*x:grid*(x+1), c])
    return output

out = max_pooling(img, 8)

# 保存して確認する
cv2.imwrite("training_IMG/training_08.png", out)

In [ ]:
%reset -f

## Q.9. ガウシアンフィルタ

ガウシアンフィルタ(3x3、標準偏差1.3)を実装し、*imori_noise.jpg*のノイズを除去せよ。

ガウシアンフィルタとは画像の**平滑化**（滑らかにする）を行うフィルタの一種であり、**ノイズ除去**にも使われる。

ノイズ除去には他にも、メディアンフィルタ(Q.10)、平滑化フィルタ(Q.11)、LoGフィルタ(Q.19)などがある。

ガウシアンフィルタは注目画素の周辺画素を、ガウス分布による重み付けで平滑化し、次式で定義される。
このような重みは**カーネル**や**フィルタ**と呼ばれる。

ただし、画像の端はこのままではフィルタリングできないため、画素が足りない部分は0で埋める。これを**0パディング**と呼ぶ。
かつ、重みは正規化する。(sum g = 1)

重みはガウス分布から次式になる。

<img src='assets/gaussian_filter.png' width=200>

```bash
重み g(x,y,s) = 1/ (2 * pi * sigma * sigma) * exp( - (x^2 + y^2) / (2*s^2))
標準偏差s = 1.3による8近傍ガウシアンフィルタは
            1 2 1
K =  1/16 [ 2 4 2 ]
            1 2 1
```

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori_noise.jpg").astype(np.float32)

def GaussianFilter(image, sigma, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # カーネルを用意する
    kernel = np.zeros((kernel_size, kernel_size), dtype=float)
    for x in range(-pad, -pad + kernel_size):
        for y in range(-pad, -pad + kernel_size):
            kernel[y + pad, x + pad] = np.exp( -(x ** 2 + y ** 2) / (2 * (sigma ** 2)))
    kernel /= (2 * np.pi * sigma * sigma)
    kernel /= kernel.sum()
    print(kernel)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.sum(kernel*tmp[y:y+kernel_size, x:x+kernel_size, c])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = GaussianFilter(img, 1.3, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_09.png", img_)

In [ ]:
%reset -f

## Q.10 メディアンフィルタ

メディアンフィルタ(3x3)を実装し、*imori_noise.jpg*のノイズを除去せよ。

メディアンフィルタとは画像の平滑化を行うフィルタの一種である。

これは注目画素の3x3の領域内の、メディアン値(中央値)を出力するフィルタである。
これもゼロパディングせよ。

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori_noise.jpg").astype(np.float32)

def MedianFilter(image, sigma, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.median(tmp[y:y+kernel_size, x:x+kernel_size, c])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MedianFilter(img, 1.3, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_10.png", img_)